### Load & Format Data

In [1]:
import cv2
import glob
import pickle
import numpy as np
import pandas as pd

##pickle for serialization
##glob searches for files that match a specific pattern


### Function to Resize Data

In [2]:
# function to read and resize images, get labels and store them into np array
def get_image_label_resize(label, filelist, dim=(32, 32), dataset='Train'):
    x = np.array([cv2.resize(cv2.imread(fname), dim, interpolation=cv2.INTER_AREA) for fname in filelist])
    y = np.array([label] * len(filelist))
        
    #print('{} examples loaded for label {}'.format(x.shape[0], label))
    return (x, y)  

### Set Up Variables

In [3]:
# data for label 0 
# images that represent the first sign of 43 
filelist = glob.glob(r"C:\Users\ally_\Downloads\Traffic-Sign-Classifier-master\Train\0\*.png")
trainx, trainy = get_image_label_resize(0, filelist)

### Resize and Load Remaining Training Images

In [ ]:
# go through remaining folder labels and store images into np array
for label in range(1,43):
   
    filelist = glob.glob(r"C:\Users\ally_\Downloads\Traffic-Sign-Classifier-master\Train\\" + str(label) + r"\*.png")
    x, y = get_image_label_resize(label, filelist)
    trainx = np.concatenate((trainx ,x))
    trainy = np.concatenate((trainy ,y))

### Serializes Training Data

In [ ]:
# save data into a pickle to later use
# Serialization
trainx.dump('../trainx.npy')
trainy.dump('../trainy.npy')

### Reads CSV, Resizes Images and Dumps Images and Corresponding ID's

In [ ]:
# get path for test images
testfile = pd.read_csv(r"C:\Users\ally_\Downloads\Traffic-Sign-Classifier-master\Test.csv")
file_names = list(testfile['Path'])

#resizes and dumps test images
X_test = np.array([cv2.resize(cv2.imread(fname), (32, 32), interpolation = cv2.INTER_AREA) for fname in file_names])
X_test.dump('../testx.npy')

#dumps images corresponding Class ID
y_test = np.array(pd.read_csv('Test.csv')['ClassId'])
y_test.dump('../testy.npy')

### Deserializes Data

In [ ]:
# load data from pickle
# Deserialization
trainx = np.load('../Trainx.npy', allow_pickle=True)
trainy = np.load('../Trainy.npy', allow_pickle=True)

In [ ]:
# load data from pickle
X_test = np.load('../testx.npy', allow_pickle=True)
y_test = np.load('../testy.npy', allow_pickle=True)

### Creates Validation Set
#### Training: Learning Model
#### Validation: Prevents Overvitting

In [ ]:
# shuffle training data and split them into training and validation
indices = np.random.permutation(trainx.shape[0])
# 20% to val
split_idx = int(trainx.shape[0]*0.8)
train_idx, val_idx = indices[:split_idx], indices[split_idx:]
X_train, X_validation = trainx[train_idx,:], trainx[val_idx,:]
y_train, y_validation = trainy[train_idx], trainy[val_idx]

### Displays Data Information

In [ ]:
# get overall stat of the whole dataset
n_train = X_train.shape[0]
n_validation = X_validation.shape[0]
n_test = X_test.shape[0]
image_shape = X_train[0].shape
n_classes = len(np.unique(y_train))
print("There are {} training examples ".format(n_train))
print("There are {} validation examples".format(n_validation))
print("There are {} testing examples".format(n_test))
print("Image data shape is {}".format(image_shape))
print("There are {} classes".format(n_classes))

### Converts to Grayscale and Normalizes
#### Removes unnecessary data "cleans it"

In [ ]:
# convert the images to grayscale
X_train_gry = np.sum(X_train/3, axis=3, keepdims=True)
X_validation_gry = np.sum(X_validation/3, axis=3, keepdims=True)
X_test_gry = np.sum(X_test/3, axis=3, keepdims=True)

# Normalize data
X_train_normalized_gry = (X_train_gry-128)/128
X_validation_normalized_gry = (X_validation_gry-128)/128
X_test_normalized_gry = (X_test_gry-128)/128

### Displays Difference in Images 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-colorblind')

# descriptions for each label
sign = pd.read_csv('signnames.csv')

# pick an image, display the original and the normalized gray image
index = np.random.randint(0, n_train)
fig, ax = plt.subplots(1,2)
ax[0].set_title('original ' + sign.loc[sign['ClassId'] ==y_train[index], 'SignName'].values[0])
ax[0].imshow(cv2.cvtColor(X_train[index], cv2.COLOR_BGR2RGB))

ax[1].set_title('norm_gry ' + sign.loc[sign['ClassId'] ==y_train[index], 'SignName'].values[0])
ax[1].imshow(X_train_normalized_gry[index].squeeze(), cmap='gray')

In [ ]:
# update the train, val and test data with normalized gray images
X_train = X_train_normalized_gry
X_validation = X_validation_normalized_gry
X_test = X_test_normalized_gry

### Start of Learning
##### Activation Function(s): ReLU (rectified linear unit), Softmax (Output Layer)
##### Conv2D: slides weights over data and provides matrix multiplication
##### MaxPooling2D:https://keras.io/api/layers/pooling_layers/max_pooling2d/
##### Flatten: flattens input, doesn't affect batch size
##### Dense: each of the neurons of the dense layers receives input from all neurons of the previous layer
##### Dropout:  for reducing over-fitting in neural network models

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

model = models.Sequential()
# Conv 32x32x1 => 28x28x6.
model.add(layers.Conv2D(filters = 6, kernel_size = (5, 5), strides=(1, 1), padding='valid', 
                        activation='relu', data_format = 'channels_last', input_shape = (32, 32, 1)))
# Maxpool 28x28x6 => 14x14x6
model.add(layers.MaxPooling2D((2, 2)))
# Conv 14x14x6 => 10x10x16
model.add(layers.Conv2D(16, (5, 5), activation='relu'))
# Maxpool 10x10x16 => 5x5x16
model.add(layers.MaxPooling2D((2, 2)))
# Flatten 5x5x16 => 400
model.add(layers.Flatten())
# Fully connected 400 => 120
model.add(layers.Dense(120, activation='relu'))
# Fully connected 120 => 84
model.add(layers.Dense(84, activation='relu'))
# Dropout
model.add(layers.Dropout(0.2))
# Fully connected, output layer 84 => 43
model.add(layers.Dense(43, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# specify optimizer, loss function and metric
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# training batch_size=128, epochs=10
conv = model.fit(X_train, y_train, batch_size=128, epochs=10, 
                    validation_data=(X_validation, y_validation))

In [ ]:
acc = [conv.history['accuracy'], conv.history['val_accuracy']]
loss = [conv.history['loss'], conv.history['val_loss']]

epoch = range(10)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epoch, acc[0], label='Training Accuracy')
plt.plot(epoch, acc[1], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epoch, loss[0], label='Training Loss')
plt.plot(epoch, loss[1], label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.evaluate(x=X_test, y=y_test)

In [ ]:
index = np.random.randint(0, n_test)
im = X_test[index]
fig, ax = plt.subplots()
ax.set_title(sign.loc[sign['ClassId'] ==np.argmax(model.predict(np.array([im]))), 'SignName'].values[0])
ax.imshow(im.squeeze(), cmap = 'gray')